In [1]:
from azureml.train.hyperdrive import BayesianParameterSampling, choice, uniform
from azureml.train.hyperdrive import BanditPolicy
from azureml.core import Workspace
from azureml.core import Experiment, Environment
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig



In [2]:
ws = Workspace.from_config()

In [3]:
param_space = {
                 '--n_estimators': choice(1,5,10, 50, 100,500),
                 '--max_depth': choice(range(1,3))
              }

#Bayesian tries to select parameter combinations that will result in improved performance from the previous selection
param_sampling = BayesianParameterSampling(param_space)


#bandit policy to stop a run if the target performance metric underperforms the best run so far by a specified margin

early_termination_policy = BanditPolicy(slack_amount = 0.2,
                                        evaluation_interval=1,
                                        delay_evaluation=5)

In [4]:
cpu_cluster_name = "cpuclst"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2', 
                                                           idle_seconds_before_scaledown=3600,
                                                           min_nodes=0, 
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

aml_run_config = RunConfiguration()
aml_run_config.target = cpu_cluster

curated_environment = Environment.get(workspace=ws,name="sklearn_20_2")
aml_run_config.environment = curated_environment

script_config = ScriptRunConfig(source_directory='./', script='train.py',run_config=aml_run_config)




Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:

hyperdrive = HyperDriveConfig(run_config=script_config,
                              hyperparameter_sampling=param_sampling,
                              policy=None,
                              primary_metric_name='Accuracy',
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=6,
                              max_concurrent_runs=4)

experiment = Experiment(workspace = ws, name = 'hyperdrive_training')
hyperdrive_run = experiment.submit(config=hyperdrive)
hyperdrive_run.wait_for_completion()

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:40.


In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])